In [ ]:
#!/usr/bin/env python3
import nest_asyncio
nest_asyncio.apply()  # Allow asyncio.run() inside Jupyter environments

import asyncio
import asyncpg
import re
import ssl
import time
import random
import string
import math
from datetime import datetime
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeoutError

# ---------------------------
# Oxylabs proxy credentials
OX_USERNAME = "cyber001_pzbfZ"       # Your Oxylabs username
OX_PASSWORD = "qweASDzxc123+"        # Your Oxylabs password
OX_PROXY = "ddc.oxylabs.io:8000"       # Oxylabs proxy server (port 8000)

# ---------------------------
# Helper functions for human-like interactions

def generate_random_email():
    """Generate a random email address for more natural interactions."""
    domains = ["gmail.com", "outlook.com", "yahoo.com", "hotmail.com"]
    username_part = ''.join(random.choices(string.ascii_lowercase + string.digits, k=random.randint(6, 10)))
    domain = random.choice(domains)
    return f"{username_part}@{domain}"

def generate_random_uk_phone():
    """Generate a random UK mobile phone number in the format 07XXXXXXXXX."""
    return "07" + ''.join(random.choices("0123456789", k=9))

def generate_random_postcode():
    """Generate a random UK postcode."""
    postcodes = ["BD3 7HR", "LS1 4AX", "M1 1AE", "B1 1HQ", "EC1A 1BB", "SW1A 1AA"]
    return random.choice(postcodes)

async def human_type(page, selector, text, random_delay=False):
    """Type text into an input field with minimal human-like delays."""
    await page.click(selector)
    await asyncio.sleep(0.1)  # Consistent small delay
    await page.type(selector, text, delay=50)  # 50ms delay between keystrokes
    if random_delay:
        await asyncio.sleep(random.uniform(0.1, 0.3))

async def human_click(page, selector, wait_after=False):
    """Click an element with a consistent delay afterward."""
    try:
        await page.click(selector)
        if wait_after:
            await asyncio.sleep(0.2)
    except Exception as e:
        print(f"Click error on {selector}: {e}")
        try:
            await page.evaluate(f'document.querySelector("{selector}").click()')
        except Exception:
            raise

async def human_scroll(page, direction, distance):
    """Scroll the page by a given distance."""
    if direction == "down":
        await page.evaluate(f'window.scrollBy(0, {distance})')
    else:
        await page.evaluate(f'window.scrollBy(0, -{distance})')
    await asyncio.sleep(0.2)

async def human_mouse_move(page, selector):
    """Move the mouse to an element (simplified)."""
    try:
        await page.hover(selector)
    except Exception:
        pass

async def simulate_human_behavior(page):
    """
    Simulate additional human behavior by scrolling, random mouse hovering,
    and taking a brief 'thinking pause.'
    """
    # Random scroll up or down between 100 and 300 pixels
    scroll_distance = random.randint(100, 300)
    direction = random.choice(["up", "down"])
    await human_scroll(page, "down" if direction == "down" else "up", scroll_distance)
    
    # Randomly hover over one of a set of common selectors (if they exist)
    potential_selectors = ["header", "nav", "footer", "img", "article"]
    selector = random.choice(potential_selectors)
    try:
        await human_mouse_move(page, selector)
    except Exception:
        pass
    
    # Pause as if reading the page (between 1 and 3 seconds)
    await asyncio.sleep(random.uniform(1.0, 3.0))

async def check_for_car_not_found(page):
    """
    Check for car not found error using multiple approaches to ensure detection
    even if the element is hidden.
    """
    # Method 1: Check for specific h1 element by class
    error_heading = await page.query_selector('h1.text-focus.ng-star-inserted')
    if error_heading:
        error_text = await page.evaluate('(element) => element.textContent', error_heading)
        if error_text and "sorry" in error_text.lower() and "find your car" in error_text.lower():
            return True
    # Method 2: Check entire page content for error message
    page_content = await page.content()
    if "sorry, we couldn't find your car" in page_content.lower():
        return True
    # Method 3: Use JavaScript to find hidden elements
    has_error = await page.evaluate('''
        () => {
            const errorHeadings = document.querySelectorAll('h1.text-focus');
            for (const heading of errorHeadings) {
                if (heading.textContent.toLowerCase().includes("sorry") && 
                    heading.textContent.toLowerCase().includes("find your car")) {
                    return true;
                }
            }
            const allElements = document.querySelectorAll('*');
            for (const element of allElements) {
                if (element.textContent.toLowerCase().includes("sorry, we couldn't find your car")) {
                    return true;
                }
            }
            return false;
        }
    ''')
    return has_error

# ---------------------------
# Exception and Parsing

class ValuationError(Exception):
    def __init__(self, message):
        super().__init__(message)
        self.message = message

def parse_valuation(valuation_text):
    """
    Extracts a numeric value from the valuation text.
    """
    cleaned_text = re.sub(r'[^\d.,]', '', valuation_text)
    cleaned_text = cleaned_text.replace(',', '')
    try:
        return float(cleaned_text)
    except ValueError:
        return None

# ---------------------------
# Main processing function

async def process_row(plate, mileage):
    """
    Use Playwright to interact with the valuation website and extract the valuation text.
    Incorporates human-like behavior to avoid bot detection.
    """
    if mileage == 0 or mileage is None:
        mileage = 100000
    elif mileage < 1000 and mileage > 0:
        # If mileage is a small number (2 or 3 digits), multiply by 1000
        # For example: 153 becomes 153000, 23 becomes 23000
        mileage = mileage * 1000
        print(f"Small mileage detected, converted to {mileage}")

    browser = None
    total_bytes = 0  # Initialize bandwidth counter
    
    try:
        async with async_playwright() as p:
            # Launch browser - default to headless mode WITHOUT proxy for reliability
            browser = await p.chromium.launch(
                headless=False,  # Set to False to show the browser window
                # Uncomment the proxy settings below if you want to use Oxylabs
                # proxy={
                #     "server": f"http://{OX_PROXY}",
                #     "username": OX_USERNAME,
                #     "password": OX_PASSWORD
                # }
            )
            
            context = await browser.new_context(
                viewport={'width': 1366, 'height': 768},
                user_agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"
            )
            await context.set_extra_http_headers({"Accept-Language": "en-GB,en;q=0.9"})
            page = await context.new_page()
            
            # RESPONSE MONITORING: Listen for network responses
            async def log_response(response):
                nonlocal total_bytes
                try:
                    size = int(response.headers.get("content-length", 0))
                    if size == 0 and response.status not in [301, 302, 303, 307, 308]:
                        try:
                            body = await response.body()
                            size = len(body)
                        except Exception:
                            pass
                    total_bytes += size
                except Exception:
                    pass
            page.on("response", log_response)
            
            # Set shorter timeouts to avoid script hanging
            page.set_default_timeout(15000)  # 15 seconds instead of 30
            page.set_default_navigation_timeout(20000)  # 20 seconds instead of 30
            
            await page.goto("https://www.webuyanycar.com/")
            await asyncio.sleep(0.3)  # Reduced from 1.0 second to 0.3 seconds
            
            # Minimize human simulation before cookie acceptance to speed things up
            # await simulate_human_behavior(page)  # Commented out to speed up cookie acceptance
            
            try:
                cookie_button = await page.wait_for_selector("#onetrust-accept-btn-handler", timeout=5000)
                if cookie_button:
                    await page.click("#onetrust-accept-btn-handler")
                    await asyncio.sleep(0.2)  # Reduced from 0.5 seconds to 0.2 seconds
            except Exception:
                pass
            
            # Reduce post-cookie simulation to a shorter pause
            # await simulate_human_behavior(page)  # Commented out full simulation
            await asyncio.sleep(0.5)  # Short pause instead of full simulation
            
            # Simulate human behavior before clicking the valuation button
            await simulate_human_behavior(page)
            
            # ----- PAGE VARIATION HANDLING -----
            max_attempts = 3
            attempts = 0
            while attempts < max_attempts:
                attempts += 1
                if await check_for_car_not_found(page):
                    print(f"Car not found (during page variation handling): {plate}")
                    return None
                
                # Try multiple selectors for registration and mileage fields
                reg_field = await page.query_selector("#vehicleReg, input[placeholder*='registration'], input[name*='reg']")
                mileage_field = await page.query_selector("#Mileage, input[placeholder*='mileage'], input[name*='mileage']")
                
                if reg_field and mileage_field:
                    print(f"Standard page with both fields detected for {plate}")
                    break
                elif reg_field and not mileage_field:
                    print(f"Variant page with only reg field detected for {plate} (attempt {attempts})")
                    await human_type(page, "#vehicleReg", plate)
                    button_clicked = False
                    for btn_selector in ['button:has-text("Get my car valuation")', 'button[type="submit"]']:
                        if await page.query_selector(btn_selector):
                            await page.click(btn_selector)
                            button_clicked = True
                            print(f"Clicked {btn_selector} for {plate}")
                            break
                    if not button_clicked:
                        form = await page.query_selector('form')
                        if form:
                            await page.evaluate('document.querySelector("form").submit()')
                            button_clicked = True
                            print(f"Submitted form for {plate}")
                    if not button_clicked:
                        print(f"Warning: Could not find button to click for {plate}")
                    await asyncio.sleep(random.uniform(2, 4))
                    # Simulate a human reading the page after form submission
                    await simulate_human_behavior(page)
                else:
                    print(f"Unexpected page state for {plate} - no reg field found")
                    await page.screenshot(path=f"unexpected_page_{plate}.png")
                    await page.reload()
                    await asyncio.sleep(random.uniform(1.5, 3.0))
            
            if attempts >= max_attempts:
                print(f"Exceeded maximum attempts ({max_attempts}) for {plate}")
                return None
            
            if await check_for_car_not_found(page):
                print(f"Car not found (before standard flow): {plate}")
                return None
            
            # Standard flow: Fill in registration and mileage
            await human_type(page, "#vehicleReg", plate)
            await human_type(page, "#Mileage", str(int(mileage)))
            # Simulate a brief pause before clicking the valuation button
            await asyncio.sleep(random.uniform(0.5, 1.5))
            await page.click("#btn-go")
            await asyncio.sleep(random.uniform(2, 4))
            if await check_for_car_not_found(page):
                print(f"Car not found after form submission: {plate}")
                return None
            
            # Simulate human behavior before filling contact form
            await simulate_human_behavior(page)
            
            # Fill out the contact form
            await page.fill("#EmailAddress", generate_random_email())
            await page.fill("#Postcode", generate_random_postcode())
            await page.fill("#TelephoneNumber", generate_random_uk_phone())
            
            try:
                survey_selector = await page.query_selector("#VehicleDetailsSurvey")
                if survey_selector:
                    await page.select_option("#VehicleDetailsSurvey", str(random.randint(1, 5)))
            except Exception:
                pass
            
            # Handle VAT section
            try:
                vat_section = await page.query_selector('label[for="IsVatRegistered"]')
                if vat_section:
                    print(f"VAT section found for {plate}")
                    for selector in ['label[for="IsVatRegisteredtrue"]', '#IsVatRegisteredtrue']:
                        if await page.query_selector(selector):
                            await page.click(selector)
                            print(f"Selected Yes using {selector}")
                            break
                    try:
                        await page.evaluate('''
                            let radio = document.querySelector('#IsVatRegisteredtrue');
                            if (radio) {
                                radio.checked = true;
                                radio.dispatchEvent(new Event('change', { bubbles: true }));
                            }
                        ''')
                    except Exception:
                        pass
                else:
                    print(f"No VAT section for {plate}, continuing")
            except Exception as e:
                print(f"Error handling VAT section: {e}")
            
            # Click advance button using multiple selectors
            try:
                for selector in ["#advance-btn", 'button:has-text("Show my valuation")', 'button[type="submit"]']:
                    if await page.query_selector(selector):
                        await page.click(selector)
                        print(f"Clicked {selector} for {plate}")
                        break
            except Exception as e:
                print(f"Error clicking advance button: {e}")
                if await check_for_car_not_found(page):
                    print(f"Car not found after advance button error: {plate}")
                    return None
                raise
            
            await asyncio.sleep(2.0)
            
            # Extract valuation using multiple selectors
            try:
                await page.wait_for_selector("div.amount, div.price, .valuation-amount", state="attached", timeout=30000)
                valuation_text = None
                for selector in ["div.amount", "div.price", ".valuation-amount", ".car-value"]:
                    element = await page.query_selector(selector)
                    if element:
                        valuation_text = await element.inner_text()
                        if valuation_text and valuation_text.strip():
                            break
                if not valuation_text:
                    valuation_text = await page.evaluate('''
                        () => {
                            const elems = document.querySelectorAll('div, span, h1, h2, h3, h4');
                            for (const el of elems) {
                                const text = el.innerText || el.textContent;
                                if (text && text.includes('£') && /\\d/.test(text)) {
                                    return text;
                                }
                            }
                            return null;
                        }
                    ''')
                if valuation_text:
                    print(f"Valuation for {plate}: {valuation_text.strip()}")
                    print(f"Total bandwidth used for listing {plate}: {total_bytes} bytes")
                    return valuation_text.strip()
                else:
                    await page.screenshot(path=f"no_valuation_{plate}.png")
                    raise ValuationError("No valuation found on page")
            except PlaywrightTimeoutError:
                await page.screenshot(path=f"timeout_{plate}.png")
                if await check_for_car_not_found(page):
                    print(f"Car not found after timeout: {plate}")
                    return None
                raise ValuationError("Timeout waiting for valuation")
    
    except Exception as e:
        if not isinstance(e, ValuationError):
            print(f"Unexpected error for {plate}: {str(e)}")
            raise ValuationError(f"Unexpected error: {str(e)}")
        raise
    finally:
        if browser:
            try:
                await browser.close()
                print(f"Browser closed for {plate}")
            except Exception as e:
                print(f"Error closing browser: {e}")

# ---------------------------
# Database functions

async def insert_failure(conn, unique_id, number_plate, mileage, failure_reason):
    try:
        await conn.execute(
            """
            INSERT INTO car_pipeline.failed_valuations (
                unique_id, number_plate, mileage, failure_reason, failed_at
            )
            VALUES ($1, $2, $3, $4, CURRENT_TIMESTAMP)
            ON CONFLICT (unique_id) DO UPDATE 
            SET number_plate = EXCLUDED.number_plate,
                mileage = EXCLUDED.mileage,
                failure_reason = EXCLUDED.failure_reason,
                failed_at = CURRENT_TIMESTAMP
            """,
            unique_id, number_plate, mileage, failure_reason
        )
        print(f"Added to failed_valuations: {unique_id}")
    except Exception as e:
        print(f"ERROR inserting failure record: {e}")

async def verify_record_exists(conn, unique_id):
    try:
        row = await conn.fetchrow(
            "SELECT unique_id FROM car_pipeline.valid_valuation WHERE unique_id = $1",
            unique_id
        )
        if row:
            print(f"✓ VERIFIED: Record {unique_id} exists in valid_valuation table")
            return True
        else:
            print(f"✗ ERROR: Record {unique_id} is NOT in valid_valuation table")
            return False
    except Exception as e:
        print(f"Error verifying record: {e}")
        return False

# ---------------------------
# Main entry point for database processing

async def process_all_entries():
    start_time = datetime.now()
    print(f"Starting WBAC valuation process at {start_time.strftime('%Y-%m-%d %H:%M:%S')}")
    
    ssl_context = ssl.create_default_context()
    ssl_context.check_hostname = False
    ssl_context.verify_mode = ssl.CERT_NONE

    dsn = (
        "postgres://postgres.jdwimnqtenkoedkfzosl:G0KUJJ8OBxJu4hsL@"
        "aws-0-eu-west-2.pooler.supabase.com:5432/postgres"
        "?pool_mode=session&sslmode=require"
    )

    conn = await asyncpg.connect(dsn=dsn, ssl=ssl_context)
    try:
        query = """
            SELECT tv.unique_id, tv.number_plate, tv.mileage, tv.ebay_url, 
                   e.salvage_category
            FROM car_pipeline.to_valuate tv
            LEFT JOIN car_pipeline.enriched_ebay_listings_auction e
            ON tv.unique_id = e.unique_id;
        """
        rows = await conn.fetch(query)
        print(f"Found {len(rows)} entries to valuate")
        processed_count = 0
        success_count = 0
        failure_count = 0

        for row in rows:
            unique_id = row['unique_id']
            plate = row['number_plate']
            mileage = row['mileage'] or 0
            original_mileage = mileage
            salvage_category = row['salvage_category']
            
            # Log salvage status if present
            if salvage_category and salvage_category in ['CAT N', 'CAT S']:
                print(f"Note: Vehicle {plate} is marked as {salvage_category}")

            print(f"\nProcessing: {plate} (ID: {unique_id})")
            try:
                valuation_text = await process_row(plate, mileage)
                if not valuation_text:
                    print(f"No valuation text returned for {plate} - likely car not found")
                    await insert_failure(conn, unique_id, plate, original_mileage, "Car not found")
                    await conn.execute("DELETE FROM car_pipeline.to_valuate WHERE unique_id = $1", unique_id)
                    failure_count += 1
                    continue

                valuation_number = parse_valuation(valuation_text)
                if valuation_number is None:
                    await insert_failure(conn, unique_id, plate, original_mileage, f"Unable to parse: {valuation_text}")
                    await conn.execute("DELETE FROM car_pipeline.to_valuate WHERE unique_id = $1", unique_id)
                    failure_count += 1
                    continue
                
                # Apply valuation adjustment for salvage category vehicles
                original_valuation = valuation_number
                if salvage_category and salvage_category in ['CAT N', 'CAT S']:
                    valuation_number = valuation_number / 2
                    print(f"Adjusting valuation for {salvage_category} vehicle: £{original_valuation} → £{valuation_number:.2f}")

                async with conn.transaction():
                    print(f"Inserting valuation for {plate}: £{valuation_number:.2f}")
                    await conn.execute(
                        """
                        INSERT INTO car_pipeline.valid_valuation (
                            unique_id, number_plate, mileage, valuation, validation_date
                        )
                        VALUES ($1, $2, $3, $4, CURRENT_TIMESTAMP)
                        ON CONFLICT (unique_id) DO UPDATE 
                        SET number_plate = EXCLUDED.number_plate,
                            mileage = EXCLUDED.mileage,
                            valuation = EXCLUDED.valuation,
                            validation_date = EXCLUDED.validation_date
                        """,
                        unique_id, plate, original_mileage, valuation_number
                    )
                    await conn.execute("DELETE FROM car_pipeline.to_valuate WHERE unique_id = $1", unique_id)
                    print(f"DB INSERT SUCCESS: {plate} valuation: £{valuation_number:.2f}" + 
                          (f" (adjusted from £{original_valuation} due to {salvage_category})" if salvage_category and salvage_category in ['CAT N', 'CAT S'] else ""))

                if await verify_record_exists(conn, unique_id):
                    success_count += 1
                else:
                    print(f"WARNING: Failed to verify record for {unique_id}")
                    failure_count += 1
            except ValuationError as ve:
                print(f"Valuation error for {plate}: {ve.message}")
                await insert_failure(conn, unique_id, plate, original_mileage, f"Valuation error: {ve.message}")
                await conn.execute("DELETE FROM car_pipeline.to_valuate WHERE unique_id = $1", unique_id)
                failure_count += 1
            except Exception as e:
                print(f"Exception for {plate}: {e}")
                await insert_failure(conn, unique_id, plate, original_mileage, f"Exception: {e}")
                await conn.execute("DELETE FROM car_pipeline.to_valuate WHERE unique_id = $1", unique_id)
                failure_count += 1

            processed_count += 1
            await asyncio.sleep(random.uniform(2.0, 5.0))
    finally:
        await conn.close()
        end_time = datetime.now()
        duration = end_time - start_time
        print(f"WBAC valuation process completed: {success_count} successful, {failure_count} failed")
        print(f"Total runtime: {duration.total_seconds():.1f} seconds")

# ---------------------------
# Single plate testing option

async def process_single_plate():
    plate = input("Enter a license plate to test: ")
    mileage_input = input("Enter the mileage (or press Enter for default 100,000): ")
    try:
        mileage = int(mileage_input) if mileage_input.strip() else 100000
    except ValueError:
        mileage = 100000
        print(f"Invalid mileage value, using default: {mileage}")
    try:
        print(f"Testing valuation for plate: {plate}, mileage: {mileage}")
        valuation_text = await process_row(plate, mileage)
        if not valuation_text:
            print("No valuation text returned - likely car not found")
            return
        valuation_number = parse_valuation(valuation_text)
        if valuation_number is None:
            print(f"Unable to parse valuation text: {valuation_text}")
            return
        print(f"SUCCESS: Valuation for {plate}: £{valuation_number}")
    except Exception as e:
        print(f"Error processing plate {plate}: {e}")

# ---------------------------
# Main entry point

if __name__ == "__main__":
    print("\nWeBuyAnyCar Valuation Tool with Human-like Behavior")
    print("===================================================")
    print("1. Process entries from database")
    print("2. Test a single plate")
    choice = input("\nEnter your choice (1 or 2): ")
    if choice == "1":
        asyncio.run(process_all_entries())
    elif choice == "2":
        asyncio.run(process_single_plate())
    else:
        print("Invalid choice, exiting.")



WeBuyAnyCar Valuation Tool with Human-like Behavior
1. Process entries from database
2. Test a single plate



Enter your choice (1 or 2):  1


Starting WBAC valuation process at 2025-05-21 01:12:01
Found 3170 entries to valuate

Processing: df15zxb (ID: 135780307989)
Standard page with both fields detected for df15zxb
No VAT section for df15zxb, continuing
Clicked #advance-btn for df15zxb
Valuation for df15zxb: £6,160
Total bandwidth used for listing df15zxb: 7533160 bytes
Browser closed for df15zxb
Inserting valuation for df15zxb: £6160.00
DB INSERT SUCCESS: df15zxb valuation: £6160.00
✓ VERIFIED: Record 135780307989 exists in valid_valuation table

Processing: sa61vuy (ID: 256931457305)
Standard page with both fields detected for sa61vuy
No VAT section for sa61vuy, continuing
Clicked #advance-btn for sa61vuy
Valuation for sa61vuy: £3,180
Total bandwidth used for listing sa61vuy: 7523423 bytes
Browser closed for sa61vuy
Inserting valuation for sa61vuy: £3180.00
DB INSERT SUCCESS: sa61vuy valuation: £3180.00
✓ VERIFIED: Record 256931457305 exists in valid_valuation table

Processing: o9mke (ID: 316817337082)
Standard page wit